In [1]:
import SpinHamiltonian as sh

# ham = sh.SpinHamiltonian('Bi')

# Bz = ham.get_field_sweep(bmin=0,bmax=1,bnum=1000)
# energy = ham.calculate_energy(Bz)

ModuleNotFoundError: No module named 'SpinHamiltonian.spinhamiltonian'